In [23]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import time
import re
import dlib

In [ ]:
# DLib Face Detection
# predictor_path = "shape_predictor_68_face_landmarks.dat"
predictor_path=r"F:\CA2b\micro-expression-recognition\CASME-SQUARE\shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)
detector = dlib.get_frontal_face_detector()

In [ ]:
class TooManyFaces(Exception):
    pass

class NoFaces(Exception):
    pass

def get_landmark(img):
    rects = detector(img, 1)
    if len(rects) > 1:
        pass
    if len(rects) == 0:
        pass
    ans = numpy.matrix([[p.x, p.y] for p in predictor(img, rects[0]).parts()])
    return ans

def annotate_landmarks(img, landmarks, font_scale = 0.4):
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(img, str(idx), pos, fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, fontScale=font_scale, color=(0, 0, 255))
        cv2.circle(img, pos, 3, color=(0, 255, 255))
    return img



In [24]:
def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

In [25]:
numbers=re.compile(r'(\d+)')
def numericalsort(value):
    parts=numbers.split(value)
    parts[1::2]=map(int, parts[1::2])
    return parts


In [26]:
def lbp_calculated_pixel_x_y(img, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val   

In [27]:
# calculate orthogonal LBPx_y for all pixels
def lbp_calculated_pixel_ortho_x_y(img_stream,frame_count, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    index=frame_count
    # for X-Y place
    center = img_stream[index][x][y]
    val_ar = []
    val_ar.append(get_pixel(img_stream[index], center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img_stream[index], center, x, y+1))       # right
    val_ar.append(get_pixel(img_stream[index], center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img_stream[index], center, x+1, y))       # bottom
    val_ar.append(get_pixel(img_stream[index], center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img_stream[index], center, x, y-1))       # left
    val_ar.append(get_pixel(img_stream[index], center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img_stream[index], center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val   


In [28]:
# calculate orthogonal LBP_x_t_for all pixels
# for X-T plane
def lbp_calculated_pixel_ortho_x_t(img_stream,frame_count, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    index=frame_count
    
    center = img_stream[index][x][y]
    val_ar = []
    # top right is x+1, y, T+1
    new_value = 0
    try:
        if img_stream[index+1][x+1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
    
    # right is x+1, y, T
    new_value = 0
    try:
        if img_stream[index][x+1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # bottom right is x+1, y, T-1
    new_value = 0
    try:
        if img_stream[index-1][x+1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # bottom is x,y, T-1
    new_value = 0
    try:
        if img_stream[index-1][x][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
        
    # bottom left is x-1, y, T-1
    new_value = 0
    try:
        if img_stream[index-1][x-1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
    
    # left is x-1,y, T
    new_value = 0
    try:
        if img_stream[index][x-1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # top left is x-1,y, T+1
    new_value = 0
    try:
        if img_stream[index+1][x-1][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # top is x,y, T+1
    new_value = 0
    try:
        if img_stream[index+1][x][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
     
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val   


In [29]:
# calculate orthogonal LBP_y_t_for all pixels
# for Y-T plane
def lbp_calculated_pixel_ortho_y_t(img_stream,frame_count, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    index=frame_count
    
    center = img_stream[index][x][y]
    val_ar = []
    # top right is x, y+1, T+1
    new_value = 0
    try:
        if img_stream[index+1][x][y+1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
    
    # right is x, y, T+1
    new_value = 0
    try:
        if img_stream[index+1][x][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # bottom right is x, y-1, T+1
    new_value = 0
    try:
        if img_stream[index+1][x][y-1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # bottom is x,y-1, T
    new_value = 0
    try:
        if img_stream[index][x][y-1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
        
    # bottom left is x, y-1, T-1
    new_value = 0
    try:
        if img_stream[index-1][x][y-1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
    
    # left is x,y, T-1
    new_value = 0
    try:
        if img_stream[index-1][x][y] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # top left is x,y+1, T-1
    new_value = 0
    try:
        if img_stream[index-1][x][y+1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)    
    
    # top is x,y+1, T
    new_value = 0
    try:
        if img_stream[index][x][y+1] >= center:
            new_value = 1
    except:
        pass
    val_ar.append(new_value)
     
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val   


In [30]:
os.chdir("F:\CA2b")
orig_dir = os.getcwd()
orig_dir

'F:\\ca2b'

In [31]:
orig_dir

'F:\\ca2b'

In [32]:
# Create the folders for each main subject
frames_source_path=os.path.join(orig_dir, 'CASME2_Extracted_Frames')
subject_list = os.listdir(frames_source_path)

for subject in subject_list:
    subject_path_list=os.path.join(frames_source_path, subject)
    file_list = os.listdir(subject_path_list)
    os.chdir("F:\CA2b\LBP_TOP_Eye")
    if not os.path.exists(subject):
        os.mkdir(subject)
os.chdir(orig_dir)


In [ ]:
start_time=time.time()
# Extract the eye and nose info from each subfolder and create directory to store info
sub_list=subject_list[0:1]
frames_source_path=os.path.join(orig_dir, 'CASME2_Extracted_Frames')
subject_list = os.listdir(frames_source_path)
for subject in sub_list:
    subject_path_list=os.path.join(frames_source_path, subject)
    file_list = os.listdir(subject_path_list)
    sub_file_list=file_list[0:1]
    for folder in sub_file_list:
        start_time=time.time()
        folder_file=os.path.join(subject_path_list, folder)
        frame_list=os.listdir(folder_file)
        # extract vectors
        # Get the frame volume sequence
        count=0
        image_sequence=[]
        for image in sorted(frame_list, key=numericalsort):
            image_file=os.path.join(folder_file, image)
            img_bgr = cv2.imread(image_file)
            landmarks = get_landmark(img_bgr)
            numpylandmarks = numpy.asarray(landmarks)
            eye_image = img_bgr[numpylandmarks[19][1]:numpylandmarks[1][1], numpylandmarks[1][0]:numpylandmarks[15][0]]
            eye_image = cv2.resize(eye_image, (32, 32), interpolation = cv2.INTER_AREA)
            eye_image = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
            image_sequence.append(eye_image)
        start_time=time.time()
        height, width = image_sequence[0].shape
        count=0
        hist_lbp_top_eye=[]
        print("sequence priocessing started for", folder)
        for image in image_sequence:
            img_lbp_x_y = np.zeros((height, width), np.uint8)
            img_lbp_x_t = np.zeros((height, width), np.uint8)
            img_lbp_y_t = np.zeros((height, width), np.uint8)
            for i in range(0, height):
                for j in range(0, width):
                    img_lbp_x_y[i,j]=lbp_calculated_pixel_ortho_x_y(image_sequence,count, i, j)
                    img_lbp_x_t[i,j]=lbp_calculated_pixel_ortho_x_t(image_sequence,count, i, j)
                    img_lbp_y_t[i,j]=lbp_calculated_pixel_ortho_y_t(image_sequence,count, i, j)
            hist_lbp_x_y = cv2.calcHist([img_lbp_x_y], [0], None, [256], [0, 256])
            hist_lbp_x_t = cv2.calcHist([img_lbp_x_t], [0], None, [256], [0, 256])
            hist_lbp_y_t = cv2.calcHist([img_lbp_y_t], [0], None, [256], [0, 256])
            histogram=np.concatenate((hist_lbp_x_y, hist_lbp_x_t, hist_lbp_y_t), axis=0)
            hist_lbp_top_eye.append(histogram)
            count=count+1
        os.chdir("F:\CA2b\LBP_TOP_Eye")
        location=os.getcwd()
        new_subject_path=os.path.join(location, subject)
        os.chdir(new_subject_path)
        if not os.path.exists(folder):
            os.mkdir(folder)
        new_folder_path=os.path.join(new_subject_path, folder)
        os.chdir(new_folder_path)
        filename="hist_lbptop_eye.npy"
        save_file=os.path.join(new_folder_path, filename)
        location=save_file
        np.save(save_file,hist_lbp_top_eye)

            
            
os.chdir(orig_dir)   

current folder is  EP02_01f
current folder path is F:\ca2b\CASME2_Extracted_Frames\sub01\EP02_01f
processing image frame1.jpg in folder,  EP02_01f  for subject sub01
processing image frame2.jpg in folder,  EP02_01f  for subject sub01
processing image frame3.jpg in folder,  EP02_01f  for subject sub01
processing image frame4.jpg in folder,  EP02_01f  for subject sub01
processing image frame5.jpg in folder,  EP02_01f  for subject sub01
processing image frame6.jpg in folder,  EP02_01f  for subject sub01
processing image frame7.jpg in folder,  EP02_01f  for subject sub01
processing image frame8.jpg in folder,  EP02_01f  for subject sub01
processing image frame9.jpg in folder,  EP02_01f  for subject sub01
processing image frame10.jpg in folder,  EP02_01f  for subject sub01
processing image frame11.jpg in folder,  EP02_01f  for subject sub01
